# Segmenting and Clustering Neighborhoods in Toronto

## Assignment Part 3

### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Load data from the output of the part 2 of Segmenting and Clustering Neighborhoods in Toronto Assignments

In [2]:
df = pd.read_csv('toronto_latlon.csv', sep=';')

Quickly examine the resulting dataframe.

In [3]:
del df['Unnamed: 0']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


Number of boroughs and neighborhoods.

In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer, as shown below.

In [5]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top.

In [6]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Now let's analyze the borough "Scarborough"

In [7]:
scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


Let's get the geographical coordinates of Scarborough.

In [38]:
address = 'Scarborough'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 54.2847601, -0.4090339.


Let's visualizat Scarborough the neighborhoods in it.

In [43]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [10]:
CLIENT_ID = 'XL24KM52GVRWBU5UIKYP0F4DBORXRBT0DSC01XII0UVDXAVP' # your Foursquare ID
CLIENT_SECRET = 'DEGIYUU1SRPD0WFRNRCHPO0BXQLLHZJRTN4I2KY1DJCGMFTE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XL24KM52GVRWBU5UIKYP0F4DBORXRBT0DSC01XII0UVDXAVP
CLIENT_SECRET:DEGIYUU1SRPD0WFRNRCHPO0BXQLLHZJRTN4I2KY1DJCGMFTE


### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [11]:
scarborough_data.loc[0, 'Neighborhood']

'Rouge, Malvern'

Get the neighborhood's latitude and longitude values.

In [12]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.811525000000074, -79.19551721399995.


### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XL24KM52GVRWBU5UIKYP0F4DBORXRBT0DSC01XII0UVDXAVP&client_secret=DEGIYUU1SRPD0WFRNRCHPO0BXQLLHZJRTN4I2KY1DJCGMFTE&v=20180605&ll=43.811525000000074,-79.19551721399995&radius=1000&limit=100'

Send the GET request and examine the resutls

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c9a2e589fb6b73b7316fa6b'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.820525009000086,
    'lng': -79.18306856701678},
   'sw': {'lat': 43.80252499100006, 'lng': -79.20796586098312}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c97975582b56dcb8320ebaa',
       'name': 'Canadiana exhibit',
       'location': {'lat': 43.81796218928876,
        'lng': -79.19337359666939,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.81796218928876,
          'lng': -79.19337359666939}],
        'distance': 736,
        'cc': 'CA',
        'city': 'Toronto',
        'state': 'ON'

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374
1,Wendy's,Fast Food Restaurant,43.807448,-79.199056
2,Grizzly Bear Exhibit,Zoo Exhibit,43.817031,-79.193458
3,Lee Valley,Hobby Shop,43.803161,-79.199681


Number of venues were returned by Foursquare

In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Explore Neighborhoods in Scarborough

Let's create a function to repeat the same process to all the neighborhoods in Scarborough

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called scarborough_venues.

In [19]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


Let's check the size of the resulting dataframe

In [20]:
print(scarborough_venues.shape)
scarborough_venues.head()

(86, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Creek, Rouge Hill, Port Union",43.78573,-79.158750,Scarborough Historical Society,43.788755,-79.162438,History Museum
1,"Highland Creek, Rouge Hill, Port Union",43.78573,-79.158750,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.76569,-79.175256,The Strawberry Patch,43.764738,-79.173081,Tea Room
3,"Guildwood, Morningside, West Hill",43.76569,-79.175256,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
4,"Guildwood, Morningside, West Hill",43.76569,-79.175256,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center


Let's check how many venues were returned for each neighborhood

In [21]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,13,13,13,13,13,13
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",1,1,1,1,1,1
"Birch Cliff, Cliffside West",6,6,6,6,6,6
Cedarbrae,2,2,2,2,2,2
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",11,11,11,11,11,11
"Cliffcrest, Cliffside, Scarborough Village West",9,9,9,9,9,9
"Dorset Park, Scarborough Town Centre, Wexford Heights",4,4,4,4,4,4
"East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4


Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 52 uniques categories.


## Analyze Each Neighborhood

In [23]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,Auto Garage,Bakery,Bar,Bus Line,Bus Stop,Business Service,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,History Museum,Hobby Shop,Hong Kong Restaurant,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Metro Station,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Restaurant,Sandwich Place,Shanghai Restaurant,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Trail,Train Station,Vietnamese Restaurant,Wine Shop
0,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [24]:
scarborough_onehot.shape

(86, 53)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,Auto Garage,Bakery,Bar,Bus Line,Bus Stop,Business Service,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,History Museum,Hobby Shop,Hong Kong Restaurant,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Metro Station,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Restaurant,Sandwich Place,Shanghai Restaurant,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Trail,Train Station,Vietnamese Restaurant,Wine Shop
0,Agincourt,0.0,0.076923,0.0,0.000000,0.000000,0.00,0.153846,0.000000,0.000000,0.00,0.0,0.00,0.076923,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.076923,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.076923,0.00,0.000000,0.076923,0.076923,0.076923,0.000000,0.153846,0.076923,0.00,0.000000,0.0,0.00,0.076923,0.00
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.00
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.166667,0.00,0.166667,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.00
3,Cedarbrae,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.5,0.00,0.000000,0.00
4,"Clairlea, Golden Mile, Oakridge",0.0,0.222222,0.0,0.222222,0.000000,0.00,0.000000,0.222222,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.111111,0.000000,0.00,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.00
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.0,0.000000,0.090909,0.00,0.090909,0.090909,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.090909,0.000000,0.000000,0.00,0.090909,0.000000,0.000000,0.00,0.000000,0.0,0.090909,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.181818,0.090909,0.0,0.000000,0.00,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.00,0.090909,0.0,0.00,0.000000,0.00
6,"Cliffcrest, Cliffside, Scarborough Village West",0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.00,0.0,0.00,0.111111,0.222222,0.000000,0.111111,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.000000,0.111111,0.111111,0.0,0.000000,0.00,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.00
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.500000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.

Let's confirm the new size

In [26]:
scarborough_grouped.shape

(15, 53)

Let's print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0         Supermarket  0.15
1  Chinese Restaurant  0.15
2       Shopping Mall  0.08
3      Discount Store  0.08
4        Skating Rink  0.08


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0             Pharmacy   1.0
1          Auto Garage   0.0
2               Bakery   0.0
3  Japanese Restaurant   0.0
4    Korean Restaurant   0.0


----Birch Cliff, Cliffside West----
             venue  freq
0             Park  0.17
1         Gym Pool  0.17
2  College Stadium  0.17
3              Gym  0.17
4     Skating Rink  0.17


----Cedarbrae----
                 venue  freq
0                Trail   0.5
1           Playground   0.5
2          Auto Garage   0.0
3           Restaurant   0.0
4  Japanese Restaurant   0.0


----Clairlea, Golden Mile, Oakridge----
           venue  freq
0         Bakery  0.22
1       Bus Line  0.22
2    Coffee Shop  0.22
3   Intersection  0.11
4  Metro Station  0.11


----Clark

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Supermarket,Hong Kong Restaurant,Skating Rink,Vietnamese Restaurant,Discount Store,Shanghai Restaurant,Shopping Mall,Pool,Sushi Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Wine Shop,Hobby Shop,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment
2,"Birch Cliff, Cliffside West",Gym Pool,Gym,General Entertainment,Skating Rink,College Stadium,Park,Wine Shop,Fast Food Restaurant,Gym / Fitness Center,Grocery Store
3,Cedarbrae,Trail,Playground,Wine Shop,Discount Store,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Coffee Shop,Intersection,Soccer Field,Metro Station,Wine Shop,Fried Chicken Joint,Gym / Fitness Center,Gym


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [30]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 3, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,4.0,History Museum,Bar,Wine Shop,Fast Food Restaurant,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256,0.0,Gym / Fitness Center,Tea Room,Park,Construction & Landscaping,Wine Shop,Fast Food Restaurant,Gym Pool,Gym,Grocery Store,Golf Course
3,M1G,Scarborough,Woburn,43.768359,-79.217590,0.0,Korean Restaurant,Business Service,Coffee Shop,Park,Wine Shop,Fried Chicken Joint,Gym Pool,Gym / Fitness Center,Gym,Grocery Store
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,3.0,Trail,Playground,Wine Shop,Discount Store,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop


Finally, let's visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you

### Cluster 1

In [33]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0.0,Gym / Fitness Center,Tea Room,Park,Construction & Landscaping,Wine Shop,Fast Food Restaurant,Gym Pool,Gym,Grocery Store,Golf Course
3,Scarborough,0.0,Korean Restaurant,Business Service,Coffee Shop,Park,Wine Shop,Fried Chicken Joint,Gym Pool,Gym / Fitness Center,Gym,Grocery Store
5,Scarborough,0.0,Restaurant,Train Station,Indian Restaurant,Grocery Store,Department Store,Gym / Fitness Center,Gym,Golf Course,Gift Shop,General Entertainment
6,Scarborough,0.0,Discount Store,Coffee Shop,Department Store,Hobby Shop,Bus Line,Bus Stop,History Museum,Gym Pool,Gym / Fitness Center,Gym
7,Scarborough,0.0,Bakery,Bus Line,Coffee Shop,Intersection,Soccer Field,Metro Station,Wine Shop,Fried Chicken Joint,Gym / Fitness Center,Gym
8,Scarborough,0.0,Fast Food Restaurant,Discount Store,Liquor Store,Coffee Shop,Furniture / Home Store,Sandwich Place,Pharmacy,Pizza Place,Gym / Fitness Center,Gym
9,Scarborough,0.0,Gym Pool,Gym,General Entertainment,Skating Rink,College Stadium,Park,Wine Shop,Fast Food Restaurant,Gym / Fitness Center,Grocery Store
10,Scarborough,0.0,Bakery,Wine Shop,Gift Shop,Fast Food Restaurant,History Museum,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course
12,Scarborough,0.0,Chinese Restaurant,Supermarket,Hong Kong Restaurant,Skating Rink,Vietnamese Restaurant,Discount Store,Shanghai Restaurant,Shopping Mall,Pool,Sushi Restaurant
13,Scarborough,0.0,Pharmacy,Hobby Shop,Golf Course,Fried Chicken Joint,Shopping Mall,Coffee Shop,Chinese Restaurant,Bus Stop,Thai Restaurant,Pizza Place


### Cluster 2

In [34]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1.0,Pharmacy,Wine Shop,Hobby Shop,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment


### Cluster 3

In [35]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,2.0,Auto Garage,Convenience Store,Fast Food Restaurant,History Museum,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop


### Cluster 4

In [36]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,3.0,Trail,Playground,Wine Shop,Discount Store,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop


### Cluster 5

In [37]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4.0,History Museum,Bar,Wine Shop,Fast Food Restaurant,Gym Pool,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop
